# 1. Create database 'data_db' in HMS

In [1]:
%%sql

-- Check metastore status
--show schemas from hms;
--show tables from hms.data_db;
--drop table hms.data_db.test;
--drop schema hms.data_db;

CREATE DATABASE data_db LOCATION 's3a://lakehouse/warehouse/data_db';

24/03/29 10:23:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

# 2. Create and populate 'olympics' data model

In [30]:
EVENTS_TABLE="hms.data_db.olympic_events"
REGIONS_TABLE="hms.data_db.olympic_regions"

In [78]:
%%sh
# Wait until it downloads the dataset (5 MBs)
wget https://techtfq.com/s/Olympics_data.zip -q -O /home/iceberg/datasets/olympics/olympics.zip

In [79]:
%%sh
# Extracting csv files from zip
unzip /home/iceberg/datasets/olympics/olympics.zip -d /home/iceberg/datasets/olympics

Archive:  /home/iceberg/datasets/olympics/olympics.zip
  inflating: /home/iceberg/datasets/olympics/athlete_events.csv  
  inflating: /home/iceberg/datasets/olympics/noc_regions.csv  


In [64]:
# Since dataset uses separator character "," inside athletes' names, forcing escape is required for parsing file
events_df = spark.read\
    .option("header", True)\
    .option("escape", '\"')\
    .csv("/home/iceberg/datasets/olympics/athlete_events.csv")
events_df.show(2)

+---+---------+---+---+------+------+-----+---+-----------+----+------+---------+----------+--------------------+-----+
| ID|     Name|Sex|Age|Height|Weight| Team|NOC|      Games|Year|Season|     City|     Sport|               Event|Medal|
+---+---------+---+---+------+------+-----+---+-----------+----+------+---------+----------+--------------------+-----+
|  1|A Dijiang|  M| 24|   180|    80|China|CHN|1992 Summer|1992|Summer|Barcelona|Basketball|Basketball Men's ...|   NA|
|  2| A Lamusi|  M| 23|   170|    60|China|CHN|2012 Summer|2012|Summer|   London|      Judo|Judo Men's Extra-...|   NA|
+---+---------+---+---+------+------+-----+---+-----------+----+------+---------+----------+--------------------+-----+
only showing top 2 rows



In [60]:
events_df.writeTo(EVENTS_TABLE).createOrReplace()

In [55]:
spark.read.table(EVENTS_TABLE).show(2)

+---+---------+---+---+------+------+-----+---+-----------+----+------+---------+----------+--------------------+-----+
| ID|     Name|Sex|Age|Height|Weight| Team|NOC|      Games|Year|Season|     City|     Sport|               Event|Medal|
+---+---------+---+---+------+------+-----+---+-----------+----+------+---------+----------+--------------------+-----+
|  1|A Dijiang|  M| 24|   180|    80|China|CHN|1992 Summer|1992|Summer|Barcelona|Basketball|Basketball Men's ...|   NA|
|  2| A Lamusi|  M| 23|   170|    60|China|CHN|2012 Summer|2012|Summer|   London|      Judo|Judo Men's Extra-...|   NA|
+---+---------+---+---+------+------+-----+---+-----------+----+------+---------+----------+--------------------+-----+
only showing top 2 rows



In [29]:
regions_df = spark.read.option("header", True).csv("/home/iceberg/datasets/olympics/noc_regions.csv")
regions_df.show(2)

+---+-----------+--------------------+
|NOC|     region|               notes|
+---+-----------+--------------------+
|AFG|Afghanistan|                null|
|AHO|    Curacao|Netherlands Antilles|
+---+-----------+--------------------+
only showing top 2 rows



In [32]:
regions_df.writeTo(REGIONS_TABLE).create()

In [33]:
spark.read.table(REGIONS_TABLE).show(5)

+---+-----------+--------------------+
|NOC|     region|               notes|
+---+-----------+--------------------+
|AFG|Afghanistan|                null|
|AHO|    Curacao|Netherlands Antilles|
|ALB|    Albania|                null|
|ALG|    Algeria|                null|
|AND|    Andorra|                null|
+---+-----------+--------------------+
only showing top 5 rows



# 3. Create and populate 'sales' data model

In [67]:
SALES_TABLE="hms.data_db.sales_summary"

In [68]:
# loading parquet file from local storage
sales_df = spark.read.parquet("/home/iceberg/datasets/sales")
sales_df.show(2)

+-------------------+----------------------+--------------------+----------------+---------------------+----------------+-------------+----------------+-----------+--------+-------------------+
|ProductCategoryName|ProductSubcategoryName|         ProductName|    CustomerName|SalesTerritoryCountry|SalesOrderNumber|OrderQuantity|TotalProductCost|SalesAmount|  TaxAmt|          OrderDate|
+-------------------+----------------------+--------------------+----------------+---------------------+----------------+-------------+----------------+-----------+--------+-------------------+
|              Bikes|            Road Bikes|    Road-150 Red, 62|     Cole Watson|               Canada|         SO43697|            1|       2171.2942|    3578.27|286.2616|2010-12-29 00:00:00|
|              Bikes|        Mountain Bikes|Mountain-100 Silv...|Rachael Martinez|               France|         SO43698|            1|       1912.1544|    3399.99|271.9992|2010-12-29 00:00:00|
+-------------------+---------

In [69]:
# Create Iceberg table
sales_df.writeTo(SALES_TABLE).createOrReplace()

In [70]:
# Read Iceberg table
spark.read.table(SALES_TABLE).show(2)

+-------------------+----------------------+--------------------+----------------+---------------------+----------------+-------------+----------------+-----------+--------+-------------------+
|ProductCategoryName|ProductSubcategoryName|         ProductName|    CustomerName|SalesTerritoryCountry|SalesOrderNumber|OrderQuantity|TotalProductCost|SalesAmount|  TaxAmt|          OrderDate|
+-------------------+----------------------+--------------------+----------------+---------------------+----------------+-------------+----------------+-----------+--------+-------------------+
|              Bikes|            Road Bikes|    Road-150 Red, 62|     Cole Watson|               Canada|         SO43697|            1|       2171.2942|    3578.27|286.2616|2010-12-29 00:00:00|
|              Bikes|        Mountain Bikes|Mountain-100 Silv...|Rachael Martinez|               France|         SO43698|            1|       1912.1544|    3399.99|271.9992|2010-12-29 00:00:00|
+-------------------+---------

# 4. Create and populate dummy 'trips' data model

In [34]:
TAXIS_TABLE="hms.data_db.trips"

In [35]:
%%sql
CREATE TABLE hms.data_db.trips
(
  vendor_id bigint,
  trip_id bigint,
  trip_distance float,
  fare_amount double,
  store_and_fwd_flag string
)
PARTITIONED BY (vendor_id);

++
||
++
++

In [36]:
%%sql
INSERT INTO hms.data_db.trips
VALUES (1, 1000371, 1.8, 15.32, 'N'), (2, 1000372, 2.5, 22.15, 'N'), (2, 1000373, 0.9, 9.01, 'N'), (1, 1000374, 8.4, 42.13, 'Y');

++
||
++
++

In [37]:
%%sql
SELECT * FROM  hms.data_db.trips;

vendor_id,trip_id,trip_distance,fare_amount,store_and_fwd_flag
1,1000371,1.7999999523162842,15.32,N
1,1000374,8.399999618530273,42.13,Y
2,1000372,2.5,22.15,N
2,1000373,0.8999999761581421,9.01,N
